# Classifier training

In [102]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.stats as sm
import sklearn

# Loading the data

In [113]:
names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv('data/out.csv')

df.head()

,age,sex,chest_pain,rest_blood_pressure,cholestorol,fast_blood_sugar,rest_ecg,max_heart_rate,exercise_angina,st_depression,st_slope,major_vessels,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [114]:
y = df.iloc[:, -1].values
X = df.iloc[:, :-1]

## Split data
Splitting the data into two sets (`train` and `test`) means that the test set can be used as a new, unbiased dataset to evaluate the performance of the model against. This also helps detect the phenomenon of overfitting, where a model is extremely tailored to the training dataset, and therefore performs poorly on unseen datasets.

In this case, 80% of the samples will go to the training set, and the remaining 20 will go to the test set.

In [194]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 13) X_test: (184, 13)
y_train: (736,) y_test: (184,)


## Scale data
Variables with much broader ranges of values might disproportionately effect the final model, so standardising the data ensures that all features are proportionately weighted. 

The formula used for feature standardisation transforms each feature to have zero-mean and unit-variance:

$ \large{x' = \frac{x - \overline{x}}{\sigma}} $

Where $x$ is the original feature vector, $ \bar{x}=average(x)$ is the mean of that feature vector, and $\sigma$ is its standard deviation.

In [195]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the classifiers
The following models will be compared:

Using `sklearn`: 
- Decision tree 
- Support Vector Machine
- Logistic Regression

Using `Tensorflow`:
- Artificial Neural Network

## Neural Network

The Neural Network was built with trial and error, until the best performing network was found. 
It contains:
- 13 input nodes, which correspond to the 13 input features
- 6 nodes in the first hidden layer, which uses the `ReLU` activation function and the `uniform` bias initialiser
- 13 nodes in the second hidden layer, which also uses the `ReLU` activation function
- 1 output layer (the probability of whether a patient is sick or not)

<img src="data/nn.png" width="600" />

In [205]:
import tensorflow as tf
from tensorflow import keras

classifier = keras.Sequential()

# adding the input layer (input_dim) and the first hidden layer
classifier.add(keras.layers.Dense(6, bias_initializer = 'uniform', activation = 'relu', input_dim = 13))

# adding the second hidden layer
classifier.add(keras.layers.Dense(13, activation = 'relu'))

# adding the output layer which contains 1 neuron 
# (this is a binary classification problem)
classifier.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [206]:
# compiling the neural network
classifier.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

# fitting the model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 2s 2ms/step - loss: 0.7079 - acc: 0.5177
Epoch 2/100
736/736 [==============================] - 0s 172us/step - loss: 0.6628 - acc: 0.6277
Epoch 3/100
736/736 [==============================] - 0s 167us/step - loss: 0.6297 - acc: 0.6807
Epoch 4/100
736/736 [==============================] - 0s 180us/step - loss: 0.5970 - acc: 0.7174
Epoch 5/100
736/736 [==============================] - 0s 179us/step - loss: 0.5639 - acc: 0.7527
Epoch 6/100
736/736 [==============================] - 0s 165us/step - loss: 0.5328 - acc: 0.7745
Epoch 7/100
736/736 [==============================] - 0s 166us/step - loss: 0.5053 - acc: 0.7867
Epoch 8/100
736/736 [==============================] - 0s 166us/step - loss: 0.4828 - acc: 0.7921
Epoch 9/100
736/736 [==============================] - 0s 188us/step - loss: 0.4646 - acc: 0.7976
Epoch 10/100
736/736 [==============================] - 0s 204us/step - loss: 0.4516 - acc: 0.8057
Epoch 11/100
736/736 

736/736 [==============================] - 0s 155us/step - loss: 0.3671 - acc: 0.8451
Epoch 84/100
736/736 [==============================] - 0s 168us/step - loss: 0.3671 - acc: 0.8465
Epoch 85/100
736/736 [==============================] - 0s 165us/step - loss: 0.3666 - acc: 0.8410
Epoch 86/100
736/736 [==============================] - 0s 156us/step - loss: 0.3663 - acc: 0.8424
Epoch 87/100
736/736 [==============================] - 0s 170us/step - loss: 0.3657 - acc: 0.8424
Epoch 88/100
736/736 [==============================] - 0s 153us/step - loss: 0.3657 - acc: 0.8437
Epoch 89/100
736/736 [==============================] - 0s 152us/step - loss: 0.3656 - acc: 0.8410
Epoch 90/100
736/736 [==============================] - 0s 159us/step - loss: 0.3650 - acc: 0.8437
Epoch 91/100
736/736 [==============================] - 0s 155us/step - loss: 0.3649 - acc: 0.8397
Epoch 92/100
736/736 [==============================] - 0s 157us/step - loss: 0.3644 - acc: 0.8410
Epoch 93/100
736/736 [=

In [207]:
y_pred = classifier.predict(X_test)
y_pred = y_pred.flatten().round()

In [208]:
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')

df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

Predicted  0.0  1.0
Actual             
0           54   21
1           19   90


In [209]:
accuracy = 100 * np.sum(y_test == y_pred) / len(y_test)
print("Test accuracy:", accuracy)

y_train_pred = classifier.predict(X_train).flatten().round()
accuracy_train = 100 * np.sum(y_train == y_train_pred) / len(y_train)
print("Train accuracy:", accuracy_train)

Test accuracy: 78.26086956521739
Train accuracy: 84.23913043478261


## Decision Tree classifier

In [210]:
def get_accuracy():
    print("Sklearn accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))
    print("\nConfusion matrix (predicted vs actual)")
    print(pd.DataFrame(np.c_[sklearn.metrics.confusion_matrix(y_test, y_pred)]))  

In [211]:
from sklearn.tree import DecisionTreeClassifier  

# Create a decision tree Classifier
classifier = DecisionTreeClassifier()  

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.6847826086956522

Confusion matrix (predicted vs actual)
    0   1
0  46  29
1  29  80


## Support Vector Machine

In [212]:
from sklearn.svm import SVC

# Create an SVM Classifier
classifier = SVC(kernel='linear') # Linear Kernel

# Train the model and predict against the validation set
y_pred = predict(classifier)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.7880434782608695

Confusion matrix (predicted vs actual)
    0   1
0  54  21
1  18  91


## Logistic regression

In [213]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.7934782608695652

Confusion matrix (predicted vs actual)
    0   1
0  55  20
1  18  91
